In [1]:
print("hello world")

hello world


In [2]:
path_to_train_images = './data/train/images/'
path_to_train_csv = ('./data/train/train.csv')

path_to_test_images = './data/test/images/'
path_to_test_csv = './data/test/test.csv'

In [44]:
#importing necessary packages
import keras
import os
import shutil
from os import listdir
from os.path import isfile, join
from keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras.models import Model
from keras.models import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D, Dropout, GlobalAveragePooling1D, GlobalAveragePooling2D, Flatten, BatchNormalization, Dense


from keras.applications.inception_v3 import InceptionV3
from keras import optimizers
from keras.optimizers import *
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
import time
import datetime


In [4]:
train_images = [f for f in listdir(path_to_train_images) if isfile(join(path_to_train_images, f))]
test_images = [f for f in listdir(path_to_test_images) if isfile(join(path_to_test_images,f))]

In [5]:
#print(train_images[0])
print(test_images[0])

9733.jpg


In [6]:
import csv
def make_list_from_csv(path_to_csv):
    with open(path_to_csv,'r') as file:
        csv_reader = csv.reader(file)
        csv_contents = []
        for line in csv_reader:
            csv_contents.append(line)
        return csv_contents

train_csv_contents = make_list_from_csv(path_to_train_csv)
test_csv_contents = make_list_from_csv(path_to_test_csv)

In [7]:
print(train_csv_contents[0])
print(test_csv_contents[0])

['image_id', 'filename', 'label']
['image_id', 'filename']


In [8]:
print(train_csv_contents[1])
print(test_csv_contents[1])

['1', '1.jpg', '2']
['7201', '7201.jpg']


In [9]:
print(train_csv_contents[-1])
print(test_csv_contents[-1])

['7200', '7200.jpg', '2']
['12000', '12000.jpg']


In [30]:
#I want to group the images by the class they belong to. Then, I will write the images into subdirectories in the class they belong to. This will happen only in the training set. We do not know the classes in the test set.
def make_groupings_per_class(csv_contents,istestdir=False):
    insurance_claim_types = []
    distributions = {}
    csv_contents_sans_label = csv_contents[1:]
    for c in csv_contents_sans_label :
        #here, i want to make a small adjustment to the test set. The given test set does not have the ytest column that would help me to evaluate my model. instead, i have to rely on the predict function to blind-predict what classes will be without knowing the true labels. Therefore, it doesn't matter what class the test set belongs to, I will merge all of them into one directory just to create the image generator. 
        if istestdir:
            insur_type=1
        else:
            insur_type = int(c[-1])
        current_img_name = c[1]
        insurance_claim_types.append(insur_type)
        if insur_type in distributions.keys():
            existing_list = distributions[insur_type]
            existing_list.append(current_img_name)
        else:
            distributions[insur_type] = [current_img_name]
    return sorted(set(insurance_claim_types)), distributions
train_claim_classes, train_image_distributions = make_groupings_per_class(train_csv_contents)
test_claim_classes,test_image_distributions = make_groupings_per_class(test_csv_contents,istestdir=True)


4800
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respec

In [32]:
"""
class 1- crack, 2 - scratch, 3 - flat tire, 4 - dent, 5 - glass shatter, 6 - lamp broken
"""
#print(train_claim_classes)
#print(len(test_image_distributions))

#CREATING SUBDIRECTORIES FOR EACH OF THE CLASSES
def refile_images_per_classes(image_dictionary,istestdir=False):
    for classname,list_of_images in image_dictionary.items():
        print(len(list_of_images))
        for l in list_of_images:
            
            try:
                if istestdir:
                    shutil.copy(path_to_test_images+l, './data/test/images/class'+str(classname))
                else:
                    shutil.copy(path_to_train_images+l, './data/train/images/class'+str(classname))
                os.remove(path_to_test_images+l)
                os.remove(path_to_train_images+l)
            except FileNotFoundError:
                print('already moved the file, please check respective class sub directory')
        
#refile_images_per_classes(train_image_distributions)
refile_images_per_classes(test_image_distributions,istestdir=True)

4800
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respective class sub directory
already moved the file, please check respec

In [33]:
#preparing for preprocessing and transormation for training
rescale = 1./255
target_size = (100,100)
batch_size = 100
class_mode = 'categorical'

train_data_gen = ImageDataGenerator(rescale=rescale,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
train_generator = train_data_gen.flow_from_directory(
    path_to_train_images,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    shuffle=True)

test_data_gen = ImageDataGenerator(rescale=rescale,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
test_generator=test_data_gen.flow_from_directory(path_to_test_images,target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    shuffle=True)




Found 7200 images belonging to 6 classes.
Found 4800 images belonging to 1 classes.


In [37]:
from sklearn.utils import class_weight
import numpy as np
def get_weight(x):
    class_weight_current = class_weight.compute_class_weight('balanced',np.unique(x),x)
    return class_weight_current
class_weight_train = get_weight(train_generator.classes)
class_weight_train

/Users/shraddha/datascience/python_code/ml_experiments/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass classes=[0 1 2 3 4 5], y=[0 0 0 ... 5 5 5] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


array([7.01754386, 0.51085568, 2.24719101, 0.57720058, 1.01265823,
       1.36054422])

In [45]:

output_directory = './outputs/'
model_directory = output_directory+'model/'+time.strftime('%Y-%m-%d %H-%M-%S') + "/"
logs_directory = output_directory+'logs/'+time.strftime('%Y-%m-%d %H-%M-%S')+'/'
model_file=model_directory+"{epoch:02d}-val_acc-{val_acc:.2f}-val_loss-{val_loss:.2f}.hdf5"

In [47]:
#configuring call backs
print("callback start time ", 'Timestamp: {:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now()))
checkpoint = ModelCheckpoint(model_file,monitor='val_acc',save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss',patience=5,verbose=1,restore_best_weights=True)
tensorboard = TensorBoard(log_dir=logs_directory,batch_size=batch_size,update_freq='batch')
reduce_learning_rate = ReduceLROnPlateau(monitor='val_loss',patience=5,cooldown=2,min_lr=0.0000000001,verbose=1)


callbacks = [checkpoint, reduce_learning_rate, early_stopping, tensorboard]
print('callback finish time ', 'Timestamp: {:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now()))

callback start time  Timestamp: 2023-12-03 13:14:26
callback finish time  Timestamp: 2023-12-03 13:14:26


In [55]:
def get_conv_model():
    model = Sequential()
    model.add(Conv2D(16, (3, 3), activation='relu', padding="same", input_shape=(3,150,150)))
    model.add(Conv2D(16, (3, 3), padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3)))

    model.add(Conv2D(32, (3, 3), activation='relu', padding="same", input_shape=(3,150,150)))
    model.add(Conv2D(32, (3, 3), padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3)))

    model.add(Conv2D(64, (3, 3), activation='relu', padding="same"))
    model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3)))

    model.add(Conv2D(96, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))
    model.add(Conv2D(96, (3, 3), padding="valid", activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3)))

    model.add(Conv2D(128, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))
    model.add(Conv2D(128, (3, 3), padding="valid", activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3)))

    model.add(Flatten())

    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(2 , activation='softmax'))


    print(model.summary())
    
    return model

In [56]:

def get_model():    
    

    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

    x = base_model.output

    x = BatchNormalization()(x)
    

    predictions = Dense(2, activation='softmax')(x) 
    
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    
    for layer in base_model.layers:
        layer.trainable = False

    model.summary()
        
    return model

In [60]:
print('recording time now - ', 'Timestamp: {:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now()))
cnn_model = get_conv_model()


recording time now -  Timestamp: 2023-12-03 13:23:37


ValueError: Negative dimension size caused by subtracting 3 from 1 for '{{node max_pooling2d_9/MaxPool}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 3, 3, 1], padding="VALID", strides=[1, 3, 3, 1]](Placeholder)' with input shapes: [?,1,50,32].

In [61]:
print("Starting Training Model", 'Timestamp: {:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now()))


steps_per_epoch=len(train_generator)

optimizer=tf.keras.optimizers.Adam()
loss='categorical_crossentropy'
metrics=['accuracy']
epochs = 100


cnn_model.compile(optimizer, loss=loss, metrics=metrics)

history = cnn_model.fit_generator(
    train_generator,
    steps_per_epoch = steps_per_epoch,
    epochs=epochs,
    verbose=2,
    callbacks=callbacks,
    class_weight=class_weight)

print("Completed Model Trainning", 'Timestamp: {:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now()))

Starting Training Model Timestamp: 2023-12-03 13:23:41


NameError: name 'cnn_model' is not defined